<a href="https://colab.research.google.com/github/panzershracker/Web-scraping/blob/master/HW_les_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests, re
from pprint import pprint
from collections import Counter as counter
from bs4 import BeautifulSoup as bs4

In [0]:
url = 'https://ru.wikipedia.org/wiki/'

target = 'дерево'.capitalize()

In [0]:
r = requests.get(url+target)

soup = bs4(r.text, 'html.parser')

head = soup.h1.next.strip()

In [4]:
print(r.status_code)
print(f'Тема: {head}')

200
Тема: Дерево


In [0]:
main_text = str(soup.find_all(['p', 'li', 'span', 'a']))

In [6]:
words = counter(re.findall('[а-яА-Я]{4,}', main_text))

print(len(words))

2260


In [7]:
for i in words.most_common(20):
  print(i)

('Редактировать', 79)
('раздел', 76)
('править', 76)
('язык', 60)
('Латинский', 56)
('деревьев', 55)
('деревья', 55)
('Дерево', 52)
('дерево', 39)
('Розовые', 36)
('растений', 35)
('страница', 32)
('дерева', 30)
('видов', 30)
('Сосновые', 30)
('Слива', 30)
('Кипарисовые', 27)
('отсутствует', 27)
('Википедия', 26)
('семейства', 25)


In [11]:
links = soup.select('#mw-content-text > div > ul > li > a')

for i in links:
  print(i.get('href'))

/wiki/%D0%A1%D0%B5%D0%BA%D0%B2%D0%BE%D0%B9%D1%8F
/wiki/%D0%A0%D0%B5%D0%B4%D0%B2%D1%83%D0%B4_(%D0%BD%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D0%B0%D1%80%D0%BA)
/wiki/%D0%91%D0%B0%D0%BE%D0%B1%D0%B0%D0%B1
/wiki/%D0%9B%D0%B5%D0%B4%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2%D1%8B%D0%B9_%D0%BF%D0%B5%D1%80%D0%B8%D0%BE%D0%B4
/wiki/%D0%94%D0%B5%D0%BD%D1%8C_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%B0
/wiki/%D0%9A%D1%80%D1%83%D0%BF%D0%BD%D0%BE%D0%BC%D0%B5%D1%80%D1%8B
/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%81%D1%82%D0%B0%D1%80%D0%B5%D0%B9%D1%88%D0%B8%D1%85_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D1%8C%D0%B5%D0%B2
/wiki/%D0%94%D1%80%D0%B5%D0%B2%D0%B5%D1%81%D0%B8%D0%BD%D0%B0
/wiki/%D0%9F%D0%BE%D1%80%D0%BE%D0%BA%D0%B8_%D0%B4%D1%80%D0%B5%D0%B2%D0%B5%D1%81%D0%B8%D0%BD%D1%8B
/wiki/%D0%A2%D1%80%D0%B0%D0%B2%D1%8F%D0%BD%D1%8B%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D1%8C%D1%8F
/wiki/%D0%9B%D0%B5%D1%81
/wiki/%D0%93%D0%B5%D0%BD%D0%B5%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8_%D0%BC%D0%BE%D0%B4%D0%B8